In [1]:
import requests
import pandas as pd
import time
import os
import json
from typing import Dict, Tuple, List
from tqdm.notebook import tqdm

# ================= Configuration =================
SUBSCRIPTION_KEY = "f75479bbfce9446580b4e27e150a3608"
JSON_PATH = "../country_list.json"
OUT_DIR = "../../data_origin/UN-Comtrade_Trade_Fossil/"

os.makedirs(OUT_DIR, exist_ok=True)

YEARS = range(2005, 2024)
COMMODITIES = "2701,2709,2710,2711"
API_URL = "https://comtradeapi.un.org/data/v1/get/C/A/HS"
BATCH_SIZE = 15 

# ================= Helpers =================
def load_country_metadata(json_path: str) -> Tuple[List[str], Dict[str, str], str]:
    """
    Load country metadata and generate M49 code lists and mappings.
    
    Returns:
        (List of M49 codes, Dict mapping M49->ISO3, String of all M49 codes for partner query)
    """
    with open(json_path, 'r', encoding='utf-8') as f:
        countries = json.load(f)['countries']
    
    mapping = {str(c['UN_M49']): c['iso'] for c in countries}
    m49_list = list(mapping.keys())
    partner_str = ",".join(m49_list)
    
    return m49_list, mapping, partner_str

def fetch_batch(year: int, reporters: List[str], partners: str, mapping: Dict[str, str]) -> pd.DataFrame:
    """
    Execute API request for a batch of reporting countries.
    
    Args:
        year: Target year.
        reporters: List of reporter M49 codes.
        partners: String of comma-separated partner M49 codes.
        mapping: Dictionary to map M49 codes back to ISO3.
        
    Returns:
        pd.DataFrame: Processed data or empty DataFrame on failure/no data.
    """
    headers = {"Ocp-Apim-Subscription-Key": SUBSCRIPTION_KEY}
    params = {
        "reporterCode": ",".join(reporters),
        "partnerCode": partners,
        "period": year,
        "cmdCode": COMMODITIES,
        "flowCode": "M",
        "customsCode": "C00",
        "motCode": "0"
    }

    try:
        response = requests.get(API_URL, params=params, headers=headers, timeout=90)
        
        if response.status_code == 200:
            data = response.json().get('data')
            if data:
                df = pd.DataFrame(data)
                # Map M49 codes to ISO3
                df['reporterCode'] = df['reporterCode'].astype(str)
                df['partnerCode'] = df['partnerCode'].astype(str)
                df['iso_i'] = df['reporterCode'].map(mapping)
                df['iso_j'] = df['partnerCode'].map(mapping)
                return df
                
    except Exception:
        pass # Silently fail for individual batches to keep process running

    return pd.DataFrame()

# ================= Main Execution =================
if __name__ == "__main__":
    print(f"🚀 Start Fetching | Batch Size: {BATCH_SIZE} | Path: {OUT_DIR}")
    
    m49_list, iso_map, all_partners = load_country_metadata(JSON_PATH)
    
    # Create batches
    batches = [m49_list[i:i + BATCH_SIZE] for i in range(0, len(m49_list), BATCH_SIZE)]
    
    pbar = tqdm(YEARS, desc="Total Progress")
    
    for yr in pbar:
        pbar.set_description(f"Processing {yr}...")
        year_dfs = []
        
        for batch in batches:
            df = fetch_batch(yr, batch, all_partners, iso_map)
            
            # Filter empty DFs to prevent FutureWarning in pd.concat
            if not df.empty:
                year_dfs.append(df)
            
            time.sleep(1.5) # Rate limiting
            
        target_file = os.path.join(OUT_DIR, f"Comtrade_{yr}.csv")
        
        if year_dfs:
            df_final = pd.concat(year_dfs, ignore_index=True)
            df_final.to_csv(target_file, index=False)
        else:
            # Create empty placeholder if no data found for the year
            pd.DataFrame(columns=['year', 'iso_i', 'iso_j', 'value']).to_csv(target_file, index=False)
            
        time.sleep(2)

    print("Done")

🚀 Start Fetching | Batch Size: 15 | Path: ../../data_origin/UN-Comtrade_Trade_Fossil/


Total Progress:   0%|          | 0/19 [00:00<?, ?it/s]

C:\Users\28274\AppData\Local\Temp\ipykernel_23888\2302477388.py:108: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat(year_dfs, ignore_index=True)
C:\Users\28274\AppData\Local\Temp\ipykernel_23888\2302477388.py:108: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat(year_dfs, ignore_index=True)
C:\Users\28274\AppData\Local\Temp\ipykernel_23888\2302477388.py:108: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In

Done


In [4]:
import pandas as pd
import numpy as np
import glob
import os

RAW_DIR = "../../data_origin/UN-Comtrade_Trade_Fossil/"
# 最终输出的矩阵文件
OUT_FILE = "../../data/1-7-Fossil_Trade_Matrix_Normalized.csv"

# 自动创建输出目录
os.makedirs(os.path.dirname(OUT_FILE), exist_ok=True)


def get_sorted_pair(row: pd.Series) -> str:
    """生成无向配对键 (e.g., 'AUS_CHN')，用于合并双向贸易额"""
    p = sorted([str(row['iso_i']), str(row['iso_j'])])
    return f"{p[0]}_{p[1]}"

def process_fossil_matrix() -> pd.DataFrame:
    """
    处理流程:
    1. 读取并合并原始数据
    2. 计算无向贸易总额 (A-B + B-A)
    3. 按年份进行 0-1 归一化 (1=最强依赖)
    4. 计算历年平均值并转换为矩阵
    """
    
    # 1. 批量读取
    all_files = glob.glob(os.path.join(RAW_DIR, "Comtrade_*.csv"))
    if not all_files:
        print(f"❌ 错误：在 {RAW_DIR} 未找到文件")
        return None

    print(f"读取 {len(all_files)} 个年份文件...")
    
    df_list = []
    for f in all_files:
        df = pd.read_csv(f)
        # 兼容列名 (primaryValue 或 primary_value)
        val_col = 'primaryValue' if 'primaryValue' in df.columns else 'primary_value'
        
        subset = df[['period', 'iso_i', 'iso_j', val_col]].copy()
        subset.columns = ['year', 'iso_i', 'iso_j', 'value']
        
        # 剔除自我贸易和空值
        subset = subset[subset['iso_i'] != subset['iso_j']]
        subset.dropna(subset=['value'], inplace=True)
        
        if not subset.empty:
            df_list.append(subset)

    # 2. 基础数据聚合
    # 先合并所有年份数据
    df_all = pd.concat(df_list, ignore_index=True)
    
    # 对每对国家在每一年的 煤+油+气 进行求和
    df_agg = df_all.groupby(['year', 'iso_i', 'iso_j'])['value'].sum().reset_index()

    # 3. 构建无向网络 (Symmetrization)
    # 逻辑: Sum(A->B) + Sum(B->A)
    df_agg['pair_key'] = df_agg.apply(get_sorted_pair, axis=1)
    df_undirected = df_agg.groupby(['year', 'pair_key'])['value'].sum().reset_index()

    # 拆分回两列，构建双向记录 (确保矩阵对称)
    df_undirected[['iso_A', 'iso_B']] = df_undirected['pair_key'].str.split('_', expand=True)
    
    # 对称化：将 A-B 的值赋予 A-B 和 B-A
    df_sym = pd.concat([
        df_undirected.rename(columns={'iso_A': 'iso_i', 'iso_B': 'iso_j'}),
        df_undirected.rename(columns={'iso_B': 'iso_i', 'iso_A': 'iso_j'})
    ], ignore_index=True)

    # 4. [核心步骤] 分年份归一化 (Yearly Normalization)
    # 计算当年的最大值和最小值
    grouped = df_sym.groupby('year')['value']
    min_vals = grouped.transform('min')
    max_vals = grouped.transform('max')
    denominator = max_vals - min_vals

    # 执行归一化: (x - min) / (max - min)
    # 结果: 1 = 当年最大贸易额(最依赖), 0 = 当年最小
    df_sym['norm_score'] = np.where(
        denominator == 0,
        0.0, 
        (df_sym['value'] - min_vals) / denominator
    )

    # 5. 计算历年平均 (Average over 2005-2023)
    df_avg = df_sym.groupby(['iso_i', 'iso_j'])['norm_score'].mean().reset_index()

    # 6. 转换为矩阵
    # 获取所有涉及的国家列表以确保矩阵完整
    all_countries = sorted(list(set(df_avg['iso_i']) | set(df_avg['iso_j'])))
    
    matrix = df_avg.pivot(index='iso_i', columns='iso_j', values='norm_score')
    
    # 重索引确保 49x49 (或实际国家数) 完整性，并补0
    matrix = matrix.reindex(index=all_countries, columns=all_countries).fillna(0)
    
    # 强制对角线为0
    np.fill_diagonal(matrix.values, 0)

    # 保存
    matrix.to_csv(OUT_FILE)
    print(f"矩阵已保存至: {OUT_FILE}")
    print(f"   矩阵维度: {matrix.shape}")
    
    return matrix

# ================= ▶️ 执行 =================
matrix_result = process_fossil_matrix()

if matrix_result is not None:
    print("\n最终归一化矩阵预览 (前 8 行/列):")
    display(matrix_result.iloc[:8, :8])

读取 19 个年份文件...
矩阵已保存至: ../../data/1-7-Fossil_Trade_Matrix_Normalized.csv
   矩阵维度: (45, 45)

最终归一化矩阵预览 (前 8 行/列):


iso_j,ARG,AUS,AUT,BEL,BGR,CAN,CHL,CHN
iso_i,,,,,,,,
ARG,0.000000e+00,0.003060,3.502107e-05,0.001651,2.621548e-08,0.000891,2.424823e-02,0.010568
AUS,3.060368e-03,0.000000,6.540019e-04,0.006999,2.028907e-04,0.000039,3.662211e-03,0.293665
AUT,3.502107e-05,0.000654,0.000000e+00,0.001543,3.784409e-04,0.000039,8.257308e-07,0.000017
BEL,1.650839e-03,0.006999,1.543458e-03,0.000000,3.419339e-04,0.010986,1.361018e-04,0.001988
BGR,2.621548e-08,0.000203,3.784409e-04,0.000342,0.000000e+00,0.000100,2.169588e-07,0.000048
CAN,8.908449e-04,0.000039,3.900310e-05,0.010986,9.953953e-05,0.000000,4.288442e-03,0.033343
CHL,2.424823e-02,0.003662,8.257308e-07,0.000136,2.169588e-07,0.004288,0.000000e+00,0.000796
CHN,1.056771e-02,0.293665,1.729109e-05,0.001988,4.780442e-05,0.033343,7.964505e-04,0.000000
